In [342]:
# import packages
import pandas as pd
from datetime import datetime

In [343]:
restaurant_url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
# !wget {restaurant_url}

In [344]:
# loading the JSON dataset into a pandas dataframe
restaurant_url = 'restaurant_data.json'
restaurant_df = pd.read_json(restaurant_url)

In [345]:
# loading the country-code excel file
excel_file_path = 'Country-Code.xlsx'
country_df = pd.read_excel(excel_file_path)

<h1>The below section contains the code for creating 'restaurant.csv', 'restaurant_events.csv' and the output threshold for qn 3</h1>

In [359]:
#normalise the series data into a pandas dataframe
restaurants_data = pd.json_normalize(restaurant_df['restaurants'])

# list to store extracted values for each restaurant
restaurant = []
restaurant_events = []
rating_list = []

# iterate through each row in the dataframe
for index, row in restaurants_data.iterrows():
    # index goes from 0 to 78
    # each row when using iterrows is a series with 19 dictionaries inside
    # another for loop to loop through the 19 dictionaries
    for restaurant_dict in row:
        if isinstance(restaurant_dict, dict):  # Check if the item is a dictionary
            # question requires the following fields
            # Restaurant Id
            # Restaurant Name
            # Country
            # City
            # User Rating Votes
            # User Aggregate Rating (in float)
            # Cuisines
            # we then use .get to reference the keys and get the respective values
            
            restaurant_id = restaurant_dict.get('restaurant.R.res_id', None)
            restaurant_name = restaurant_dict.get('restaurant.name', None)
            country_id = restaurant_dict.get('restaurant.location.country_id', None)
            city = restaurant_dict.get('restaurant.location.city', None)
            user_rating_votes = restaurant_dict.get('restaurant.user_rating.votes', None)
            aggregate_rating = restaurant_dict.get('restaurant.user_rating.aggregate_rating', None)
            cuisines = restaurant_dict.get('restaurant.cuisines', None)
            rating_text = restaurant_dict.get('restaurant.user_rating.rating_text')

            
    
            #append the .get values as a dictionary to the list called restaurant
            restaurant.append({
                'Restaurant ID': restaurant_id,
                'Restaurant Name': restaurant_name,
                'Country ID': country_id,
                'City': city,
                'User Rating Votes': user_rating_votes,
                'User Aggregate Rating': aggregate_rating,
                'Cuisines': cuisines
            })
            
            #extract the events for this restaurant, if any - this is for qn2
            events = restaurant_dict.get('restaurant.zomato_events', [])
            for event in events:
                event_data = event.get('event', {})
                event_start_date = pd.to_datetime(event_data.get('start_date', None), errors='coerce')
                event_end_date = pd.to_datetime(event_data.get('end_date', None), errors='coerce')
                
                # extracting photo_url
                photos_list = event_data.get('photos','NA')
                photo_url = photos_list[0]['photo'].get('url', 'NA') if photos_list else 'NA'
        
                #filter for events that took place in April 2019
                #event must have both start and end date
                if event_start_date and event_end_date and ((event_start_date <= april_end) and (event_end_date >= april_start)): 
                    restaurant_events.append({
                    'Event Id': event_data.get('event_id', 'NA'),
                    'Restaurant Id': restaurant_dict.get('restaurant.R.res_id', 'NA'),
                    'Restaurant Name': restaurant_dict.get('restaurant.name', 'NA'),
                    'Photo URL': photo_url,
                    'Event Title': event_data.get('title', 'NA'),
                    'Event Start Date': event_start_date if event_start_date else 'NA',
                    'Event End Date': event_end_date if event_end_date else 'NA'
                    })
                    
            #append the .get values as a dictionary to the list called rating_list  
            if rating_text in ("Excellent", "Very Good", "Good", "Average", "Poor") and country_id != 17:
                rating_list.append({
                    'Rating Score': aggregate_rating,
                    'Rating Category': rating_text
                })

#------------------- Qn1 csv -----------------
# Convert the extracted list of dictionaries into a dataframe
retaurant_df = pd.DataFrame(restaurant)
# now we need to join on country id and select country name to get the required field
merged_df = pd.merge(retaurant_df, country_df, left_on='Country ID', right_on='Country Code')
#using an inner join to remove non matching columns. In the process, we removed rows with the Country ID 17 and City = "dummy" in total there were 20 such observations
#select the relevant columns
final_df = merged_df[['Restaurant ID', 'Restaurant Name', 'Country','City', 'User Rating Votes', 'User Aggregate Rating', 'Cuisines']]
final_df.to_csv('restaurants.csv', index=False)


#------------------- Qn2 csv -----------------
events_df = pd.DataFrame(restaurant_events)
events_df.fillna("NA", inplace=True)
events_df.to_csv('restaurant_events.csv', index=False)


#------------------- Qn3 output -----------------
ratings_df = pd.DataFrame(rating_list)
ratings_df['Rating Score'] = ratings_df['Rating Score'].astype(float)
thresholds = ratings_df.groupby('Rating Category')['Rating Score'].agg(['min', 'max', 'size'])
thresholds.rename(columns={'size': 'frequency', 'min': "Lower Bound", 'max': "Upper Bound"}, inplace=True)
thresholds_sorted = thresholds.sort_values('Upper Bound', ascending=False)
print(thresholds_sorted)

                 Lower Bound  Upper Bound  frequency
Rating Category                                     
Excellent                4.5          4.9        435
Very Good                4.0          4.4        623
Good                     3.5          3.9        143
Average                  2.5          3.4         60
Poor                     2.2          2.2          1
